# Passive AllenNLP SRL BERT

In [21]:
from allennlp_models.pretrained import load_predictor

In [22]:
import checklist
from checklist.editor import Editor
from checklist.perturb import Perturb
from checklist.test_types import MFT, INV, DIR
from checklist.expect import Expect

In [23]:
from checklist.pred_wrapper import PredictorWrapper

In [24]:
import csv

In [25]:
import logging
logging.getLogger('allennlp.common.params').disabled = True 
logging.getLogger('allennlp.nn.initializers').disabled = True 
logging.getLogger('allennlp.modules.token_embedders.embedding').disabled = True 
logging.getLogger('urllib3.connectionpool').disabled = True 
logging.getLogger('allennlp.common.plugins').disabled = True 
logging.getLogger('allennlp.common.model_card').disabled = True 
logging.getLogger('allennlp.models.archival').disabled = True 
logging.getLogger('allennlp.data.vocabulary').disabled = True 
logging.getLogger('cached_path').disabled = True

In [26]:
srl_predictor = load_predictor('structured-prediction-srl-bert')
output = srl_predictor.predict("Someone was cuddled by Peter.")
output

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertModel: ['cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.seq_relationship.weight', 'cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


{'verbs': [{'verb': 'was',
   'description': 'Someone [V: was] cuddled by Peter .',
   'tags': ['O', 'B-V', 'O', 'O', 'O', 'O']},
  {'verb': 'cuddled',
   'description': '[ARG1: Someone] was [V: cuddled] [ARG0: by Peter] .',
   'tags': ['B-ARG1', 'O', 'B-V', 'B-ARG0', 'I-ARG0', 'O']}],
 'words': ['Someone', 'was', 'cuddled', 'by', 'Peter', '.']}

In [27]:
def predict_srl(data):
    
    pred = []
    for d in data:
        pred.append(srl_predictor.predict(d))
    return pred

predict_and_conf = PredictorWrapper.wrap_predict(predict_srl)

In [28]:
d = ["Someone was cuddled by Peter."]
pred = predict_and_conf(d)
pred

([{'verbs': [{'verb': 'was',
     'description': 'Someone [V: was] cuddled by Peter .',
     'tags': ['O', 'B-V', 'O', 'O', 'O', 'O']},
    {'verb': 'cuddled',
     'description': '[ARG1: Someone] was [V: cuddled] [ARG0: by Peter] .',
     'tags': ['B-ARG1', 'O', 'B-V', 'B-ARG0', 'I-ARG0', 'O']}],
   'words': ['Someone', 'was', 'cuddled', 'by', 'Peter', '.']}],
 array([1.]))

In [29]:
def format_srl(x, pred, conf, label=None, meta=None):
    
    return pred['verbs'][1]['description']

In [30]:
def get_arg(pred, arg_target='ARG0'):
    predicate_arguments = pred['verbs'][1]
    #print(predicate_arguments)
    words = pred['words']
    tags = predicate_arguments['tags']
    
    arg_list = []
    for t, w in zip(tags, words):
        arg = t
        if '-' in t:
            arg = t.split('-')[1]
        if arg == arg_target:
            arg_list.append(w)
    arg_set = set(arg_list)
    return arg_set

In [31]:
def format_srl(x, pred, conf, label=None, meta=None):
    results = []
    predicate_structure = pred['verbs'][1]['description']
        
    return predicate_structure

In [32]:
def found_arg1_people(x, pred, conf, label=None, meta=None):
    
    # people should be recognized as arg1

    people = set([meta['first_name']])
    arg_1 = get_arg(pred, arg_target='ARG1')

    if arg_1 == people:
        pass_ = True
    else:
        pass_ = False
    return pass_


expect_arg1 = Expect.single(found_arg1_people)

In [33]:
# initialize editor object
editor = Editor()

# create examples ARG0
t = editor.template("{first_name} was cuddled by Peter." , meta=True,nsamples=500, remove_duplicates=True)
with open('/datasets/passive_arg1_cuddled_bert.txt', 'w') as f:
    print(t.data, file = f)

# initialize a test object
test = MFT(**t, name = 'detect_arg0_name_default_position', expect=expect_arg1)
output = test.run(predict_and_conf)
test.summary(format_example_fn=format_srl)
i = test.results['preds']
expect_for_i = test.results['expect_results']
with open ('/results/passive_arg1_cuddled_bert.csv','w') as f:
    writer = csv.writer(f)
    for result, exp in zip(i, expect_for_i):
        
        case = result['words']
        exp = exp
        writer.writerow([case, exp])
        print(case,exp)

Predicting 500 examples
Test cases:      500
Fails (rate):    0 (0.0%)
['Sara', 'was', 'cuddled', 'by', 'Peter', '.'] [ True]
['Tony', 'was', 'cuddled', 'by', 'Peter', '.'] [ True]
['Charlie', 'was', 'cuddled', 'by', 'Peter', '.'] [ True]
['Harriet', 'was', 'cuddled', 'by', 'Peter', '.'] [ True]
['Henry', 'was', 'cuddled', 'by', 'Peter', '.'] [ True]
['Michelle', 'was', 'cuddled', 'by', 'Peter', '.'] [ True]
['Thomas', 'was', 'cuddled', 'by', 'Peter', '.'] [ True]
['Alan', 'was', 'cuddled', 'by', 'Peter', '.'] [ True]
['Emily', 'was', 'cuddled', 'by', 'Peter', '.'] [ True]
['Jason', 'was', 'cuddled', 'by', 'Peter', '.'] [ True]
['Sam', 'was', 'cuddled', 'by', 'Peter', '.'] [ True]
['Amy', 'was', 'cuddled', 'by', 'Peter', '.'] [ True]
['Kevin', 'was', 'cuddled', 'by', 'Peter', '.'] [ True]
['Lucy', 'was', 'cuddled', 'by', 'Peter', '.'] [ True]
['Helen', 'was', 'cuddled', 'by', 'Peter', '.'] [ True]
['Stephanie', 'was', 'cuddled', 'by', 'Peter', '.'] [ True]
['Jean', 'was', 'cuddled', 'b

In [34]:
# initialize editor object
editor = Editor()

# create examples ARG0
t = editor.template("{first_name} was hugged by Peter." , meta=True,nsamples=500, remove_duplicates=True)
with open('/datasets/passive_arg1_hugged_bert.txt', 'w') as f:
    print(t.data, file = f)

# initialize a test object
test = MFT(**t, name = 'detect_arg0_name_default_position', expect=expect_arg1)
output = test.run(predict_and_conf)
test.summary(format_example_fn=format_srl)
i = test.results['preds']
expect_for_i = test.results['expect_results']
with open ('/results/passive_arg1_hugged_bert.csv','w') as f:
    writer = csv.writer(f)
    for result, exp in zip(i, expect_for_i):
        
        case = result['words']
        exp = exp
        writer.writerow([case, exp])
        print(case,exp)

Predicting 500 examples
Test cases:      500
Fails (rate):    0 (0.0%)
['Elizabeth', 'was', 'hugged', 'by', 'Peter', '.'] [ True]
['Jill', 'was', 'hugged', 'by', 'Peter', '.'] [ True]
['Philip', 'was', 'hugged', 'by', 'Peter', '.'] [ True]
['Roger', 'was', 'hugged', 'by', 'Peter', '.'] [ True]
['Lucy', 'was', 'hugged', 'by', 'Peter', '.'] [ True]
['Suzanne', 'was', 'hugged', 'by', 'Peter', '.'] [ True]
['Nancy', 'was', 'hugged', 'by', 'Peter', '.'] [ True]
['Emily', 'was', 'hugged', 'by', 'Peter', '.'] [ True]
['Kate', 'was', 'hugged', 'by', 'Peter', '.'] [ True]
['Jimmy', 'was', 'hugged', 'by', 'Peter', '.'] [ True]
['Frances', 'was', 'hugged', 'by', 'Peter', '.'] [ True]
['Mary', 'was', 'hugged', 'by', 'Peter', '.'] [ True]
['Adam', 'was', 'hugged', 'by', 'Peter', '.'] [ True]
['Steven', 'was', 'hugged', 'by', 'Peter', '.'] [ True]
['Carol', 'was', 'hugged', 'by', 'Peter', '.'] [ True]
['Margaret', 'was', 'hugged', 'by', 'Peter', '.'] [ True]
['Kim', 'was', 'hugged', 'by', 'Peter', '

In [35]:
# initialize editor object
editor = Editor()

# create examples ARG0
t = editor.template("{first_name} was adored by Peter." , meta=True,nsamples=500, remove_duplicates=True)


In [36]:
with open('/datasets/passive_arg1_adored_bert.txt', 'w') as f:
    print(t.data, file = f)

# initialize a test object
test = MFT(**t, name = 'detect_arg0_name_default_position', expect=expect_arg1)
output = test.run(predict_and_conf)
test.summary(format_example_fn=format_srl)
i = test.results['preds']
expect_for_i = test.results['expect_results']
with open ('/results/passive_arg1_adored_bert.csv','w') as f:
    writer = csv.writer(f)
    for result, exp in zip(i, expect_for_i):
        
        case = result['words']
        exp = exp
        writer.writerow([case, exp])
        print(case,exp) 

Predicting 500 examples
Test cases:      500
Fails (rate):    0 (0.0%)
['Margaret', 'was', 'adored', 'by', 'Peter', '.'] [ True]
['Carol', 'was', 'adored', 'by', 'Peter', '.'] [ True]
['Kathy', 'was', 'adored', 'by', 'Peter', '.'] [ True]
['Alexandra', 'was', 'adored', 'by', 'Peter', '.'] [ True]
['Ken', 'was', 'adored', 'by', 'Peter', '.'] [ True]
['Sue', 'was', 'adored', 'by', 'Peter', '.'] [ True]
['Donna', 'was', 'adored', 'by', 'Peter', '.'] [ True]
['Sharon', 'was', 'adored', 'by', 'Peter', '.'] [ True]
['Jean', 'was', 'adored', 'by', 'Peter', '.'] [ True]
['Joe', 'was', 'adored', 'by', 'Peter', '.'] [ True]
['Robin', 'was', 'adored', 'by', 'Peter', '.'] [ True]
['James', 'was', 'adored', 'by', 'Peter', '.'] [ True]
['Walter', 'was', 'adored', 'by', 'Peter', '.'] [ True]
['Amanda', 'was', 'adored', 'by', 'Peter', '.'] [ True]
['Charles', 'was', 'adored', 'by', 'Peter', '.'] [ True]
['Rachel', 'was', 'adored', 'by', 'Peter', '.'] [ True]
['Lauren', 'was', 'adored', 'by', 'Peter', 

In [37]:
verbs = ['hugged', 'kissed', 'loved', 'appreciated', 'adored', 'cuddled']

In [38]:
# initialize editor object
editor = Editor()

# create examples ARG0
t = editor.template("{first_name} was {verb} by Peter.", verb = verbs, meta=True,nsamples=500, remove_duplicates=True)
with open('/datasets/passive_arg1_verb_variation_bert.txt', 'w') as f:
    print(t.data, file = f)


# initialize a test object
test = MFT(**t, name = 'detect_arg0_name_default_position', expect=expect_arg1)
output = test.run(predict_and_conf)
test.summary(format_example_fn=format_srl)
i = test.results['preds']
expect_for_i = test.results['expect_results']
with open ('/results/passive_arg1_verb_variation_bert.csv','w') as f:
    writer = csv.writer(f)
    for result, exp in zip(i, expect_for_i):
        
        case = result['words']
        exp = exp
        writer.writerow([case, exp])
        print(case,exp) 

Predicting 500 examples
Test cases:      500
Fails (rate):    0 (0.0%)
['Mark', 'was', 'cuddled', 'by', 'Peter', '.'] [ True]
['Sara', 'was', 'cuddled', 'by', 'Peter', '.'] [ True]
['Lauren', 'was', 'cuddled', 'by', 'Peter', '.'] [ True]
['Helen', 'was', 'kissed', 'by', 'Peter', '.'] [ True]
['Gary', 'was', 'kissed', 'by', 'Peter', '.'] [ True]
['Carolyn', 'was', 'adored', 'by', 'Peter', '.'] [ True]
['Tim', 'was', 'cuddled', 'by', 'Peter', '.'] [ True]
['George', 'was', 'appreciated', 'by', 'Peter', '.'] [ True]
['Harriet', 'was', 'appreciated', 'by', 'Peter', '.'] [ True]
['Julie', 'was', 'loved', 'by', 'Peter', '.'] [ True]
['Jill', 'was', 'adored', 'by', 'Peter', '.'] [ True]
['Sally', 'was', 'cuddled', 'by', 'Peter', '.'] [ True]
['Jack', 'was', 'adored', 'by', 'Peter', '.'] [ True]
['Cynthia', 'was', 'cuddled', 'by', 'Peter', '.'] [ True]
['Stephen', 'was', 'kissed', 'by', 'Peter', '.'] [ True]
['Christopher', 'was', 'loved', 'by', 'Peter', '.'] [ True]
['Kim', 'was', 'loved', 'b

# End of Notebook